In [ ]:
!pip install einops
!pip install wandb
!pip install codecarbon
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/com

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wandb login 6d0d4fcf28a32488eb9c49d8fb9198328577975c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import os
import sys
#list the current working dir
os.getcwd()
#change the current working dir
os.chdir('/content/drive/MyDrive/HPVIT')

In [ ]:
import torch
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets.cifar import CIFAR10, CIFAR100
from torchvision.datasets.mnist import MNIST
from torchvision.datasets import VOCSegmentation
import wandb
from codecarbon import track_emissions
# VISION TRANSFORMER PROBABILISTICO CON FEATURE SELECTION
# from hvit.heuristic_vision_transformer_block import HViT
# VISION TRANSFORMER CON FEATURE SELECTION
from hvit.filtered_vision_transformer_block import FViT
# VISION TRANSFORMER 
from hvit.vision_transformer_block import ViT

# CIFAR10 Dataset Loader

In [ ]:
transform = transforms.Compose([
        # Resize the image to (64, 64)
        transforms.Resize((224, 224)),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image with mean and standard deviation of 0.5
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
# Load CIFAR10 dataset
train_set = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)
# Create data loaders
train_loader = DataLoader(train_set, shuffle=False, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

Files already downloaded and verified
Files already downloaded and verified


# CIFAR100 Dataset Loader

In [ ]:
transform = transforms.Compose([
        # Resize the image to (64, 64)
        transforms.Resize((224, 224)),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image with mean and standard deviation of 0.5
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
# Load CIFAR10 dataset
train_set = CIFAR100(root='./data', train=True, download=True, transform=transform)
test_set = CIFAR100(root='./data', train=False, download=True, transform=transform)
# Create data loaders
train_loader = DataLoader(train_set, shuffle=False, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

Files already downloaded and verified
Files already downloaded and verified


# MNIST Dataset Loader

In [ ]:
transform = transforms.Compose([
        # Resize the image to (64, 64)
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224, 224)),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image with mean and standard deviation of 0.5
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
# Load CIFAR10 dataset
train_set = MNIST(root='./data', train=True, download=True, transform=transform)
test_set = MNIST(root='./data', train=False, download=True, transform=transform)
# Create data loaders
train_loader = DataLoader(train_set, shuffle=False, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

# VOCSegmentation DATASET LOADER

In [ ]:
transform = transforms.Compose([
        # Resize the image to (64, 64)
        transforms.Resize((224, 224)),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image with mean and standard deviation of 0.5
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
# Load CIFAR10 dataset
train_set = VOCSegmentation(root='./data',year='2012',image_set='train', download=True, transform=transform)
test_set = VOCSegmentation(root='./data',year='2012',image_set='trainval', download=True, transform=transform)
# Create data loaders
train_loader = DataLoader(train_set, shuffle=False, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


# Define device

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('GPU is not available, using CPU')

Using GPU


# Run

In [ ]:
depths = 2
project_name = f"RQ2_Depths_{depths}_train"
run = "default"
dataset = "VOCSegmentation"
classes = 19

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
wandb.finish()

In [ ]:
if run != 'default':
  wandb.init(
  # set the wandb project where this run will be logged
  project=project_name,
  name=run,
  # track hyperparameters and run metadata
  config={
        "FViT": True,
        "dataset": dataset,
        "task": "Image Recognition",
        "epochs": 5,
        "learning_rate":1e-3,
        "optimizer": "Adam",
        "loss": "Cross Entropy",
        "in_channels":3,
        "patch_size":16,
        "emb_size":64,
        "img_size":224,
        "depth":depths,
        "num_heads":12,
        "n_classes":classes,
        "top_k":138,
        "heuristic": run,
        "probabilistic":True,
        "prob":0.5,
        "decay_rate":5e-3,
        "batch_size":len(train_loader),
      }
  )
else:
  wandb.init(
      # set the wandb project where this run will be logged
      project=project_name,
      name=run,
      # track hyperparameters and run metadata
      config={
        "FViT": False,
        "dataset": dataset,
        "task": "Image Recognition",
        "epochs": 5,
        "learning_rate":1e-3,
        "optimizer": "Adam",
        "loss": "Cross Entropy",
        "in_channels":3,
        "patch_size":16,
        "emb_size":64,
        "img_size":224,
        "depth":depths,
        "num_heads":12,
        "n_classes":classes,
        "batch_size":len(train_loader),
      }
  )

@track_emissions(offline=True, country_iso_code="ITA", output_file=f"./evaluation/{project_name}_{run}_{dataset}.csv")
def train():
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
    # Defining model and training parameters
    '''
    model = HViT(in_channels=3,
                 patch_size=16,
                 emb_size=64,
                 img_size=224,
                 depth=8,
                 num_heads=12,
                 n_classes=10,
                 top_k=128,
                 heuristic='contrast',
                 prob=0,
                 decay_rate=0,
                 probabilistic=True,
                 batch_size=len(train_loader),
                 verbose=False).to(device)
    '''
    if run == 'default':
      model = ViT(in_channels=3,
                patch_size=16,
                emb_size=64,
                img_size=224,
                depth=depths,
                n_classes=classes,
                num_heads=12).to(device)
    else:
      model = FViT(in_channels=3,
                 patch_size=16,
                 emb_size=64,
                 img_size=224,
                 depth=depths,
                 num_heads=12,
                 n_classes=classes,
                 top_k=138,
                 heuristic=run,
                 probabilistic=True,
                 prob=0.5,
                 decay_rate=5e-3,
                 batch_size=len(train_loader),
                 verbose=False).to(device)
       
    
    
    n_epochs = 5
    lr = 1e-3
    # Training loop
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = CrossEntropyLoss()
    
    for epoch in trange(n_epochs, desc="Training"):
      train_loss = 0.0
      for batch in train_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += loss.detach().cpu().item() / len(train_loader)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
      print(f"Epoch {epoch + 1}/{n_epochs} loss: {train_loss:.2f}")
      wandb.log({'loss': train_loss})
    return model;

model = train()
wandb.finish()

wandb: Currently logged in as: ma_li (mali_team). Use `wandb login --relogin` to force relogin


[codecarbon INFO @ 20:12:30] offline tracker init
[codecarbon INFO @ 20:12:30] [setup] RAM Tracking...
[codecarbon INFO @ 20:12:30] [setup] GPU Tracking...
[codecarbon INFO @ 20:12:30] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:12:30] [setup] CPU Tracking...
[codecarbon WARNING @ 20:12:30] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:12:32] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:12:32] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 20:12:32] >>> Tracker's metadata:
[codecarbon INFO @ 20:12:32]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:12:32]   Python version: 3.10.11
[codecarbon INFO @ 20:12:32]   CodeCarbon version: 2.2.1
[codecarbon INFO @ 20:12:32]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:12:32]   CPU count: 2
[codecarbon INFO @ 20:12:32]   CPU model: Intel(R) Xeon(R

Using device:  cuda (Tesla T4)


Training:   0%|          | 0/5 [00:02<?, ?it/s]
[codecarbon INFO @ 20:12:39] 
Graceful stopping: collecting and writing information.
Please wait a few seconds...
[codecarbon INFO @ 20:12:39] Energy consumed for RAM : 0.000009 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:12:39] Energy consumed for all GPUs : 0.000051 kWh. Total GPU Power : 25.669 W
[codecarbon INFO @ 20:12:39] Energy consumed for all CPUs : 0.000085 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:12:39] 0.000146 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:12:39] Done!



TypeError: ignored

In [ ]:
project_name = f"RQ2_Depths_{depths}_inference"

if run != 'default':
  wandb.init(
  # set the wandb project where this run will be logged
  project=project_name,
  name=run,
  # track hyperparameters and run metadata
  config={
        "FViT": True,
        "dataset": dataset,
        "task": "Image Recognition",
        "epochs": 5,
        "learning_rate":1e-3,
        "optimizer": "Adam",
        "loss": "Cross Entropy",
        "in_channels":3,
        "patch_size":16,
        "emb_size":64,
        "img_size":224,
        "depth":depths,
        "num_heads":12,
        "n_classes":classes,
        "top_k":138,
        "heuristic": run,
        "probabilistic":True,
        "prob":0.5,
        "decay_rate":5e-3,
        "batch_size":len(train_loader),
      }
  )
else:
  wandb.init(
      # set the wandb project where this run will be logged
      project=project_name,
      name=run,
      # track hyperparameters and run metadata
      config={
        "FViT": False,
        "dataset": dataset,
        "task": "Image Recognition",
        "epochs": 5,
        "learning_rate":1e-3,
        "optimizer": "Adam",
        "loss": "Cross Entropy",
        "in_channels":3,
        "patch_size":16,
        "emb_size":64,
        "img_size":224,
        "depth":depths,
        "num_heads":12,
        "n_classes":classes,
        "batch_size":len(train_loader),
      }
  )

@track_emissions(offline=True, country_iso_code="ITA", output_file=f"./evaluation/{project_name}_{run}_{dataset}.csv")
def inference():
  criterion = CrossEntropyLoss()
  with torch.no_grad():
      correct, total = 0, 0
      test_loss = 0.0
      for batch in tqdm(test_loader, desc="Testing"):
          x, y = batch
          x, y = x.to(device), y.to(device)
          y_hat = model(x)
          loss = criterion(y_hat, y)
          test_loss += loss.detach().cpu().item() / len(test_loader)

          correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
          total += len(x)
      print(f"Test loss: {test_loss:.2f}")
      wandb.log({'loss': test_loss})
      print(f"Test accuracy: {correct / total * 100:.2f}%")
      accuracy = correct / total * 100
      wandb.log({'accuracy': accuracy})

inference()
wandb.finish()

[codecarbon INFO @ 14:33:01] offline tracker init
[codecarbon INFO @ 14:33:01] [setup] RAM Tracking...
[codecarbon INFO @ 14:33:01] [setup] GPU Tracking...
[codecarbon INFO @ 14:33:01] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:33:01] [setup] CPU Tracking...
[codecarbon WARNING @ 14:33:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 14:33:02] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 14:33:02] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 14:33:03] >>> Tracker's metadata:
[codecarbon INFO @ 14:33:03]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 14:33:03]   Python version: 3.10.11
[codecarbon INFO @ 14:33:03]   CodeCarbon version: 2.2.1
[codecarbon INFO @ 14:33:03]   Available RAM : 12.678 GB
[codecarbon INFO @ 14:33:03]   CPU count: 2
[codecarbon INFO @ 14:33:03]   CPU model: Intel(R) Xeon(R

Test loss: 0.09
Test accuracy: 97.39%


accuracy,▁
loss,▁
accuracy,97.39
loss,0.08975
